<a href="https://colab.research.google.com/github/JP-SystemsX/24cast/blob/master/AI_Grid_Hackathon_JP_DEV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
!pip install -q wget
!pip install -q autogluon
!pip install -q pyarrow==15.0.2
!pip install -q pandas

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.0/428.0 kB 27.4 MB

In [2]:
!wget https://zenodo.org/records/3899018/files/LoadProfile_20IPs_2016.csv -q
!wget https://zenodo.org/records/3899018/files/LoadProfile_30IPs_2017.csv -q

In [3]:
!ls

LoadProfile_20IPs_2016.csv  LoadProfile_30IPs_2017.csv	sample_data


In [1]:
import pandas as pd
from copy import deepcopy
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from pathlib import Path

In [2]:
def train_test_split():
    lp_20ips_2016 = pd.read_csv("LoadProfile_20IPs_2016.csv", sep=";", skiprows=1)
    lp_30ips_2017 = pd.read_csv("LoadProfile_30IPs_2017.csv", sep=";", skiprows=1)
    Path("./tune").mkdir(exist_ok=True)
    Path("./test").mkdir(exist_ok=True)

    lp_20ips_2016["Time stamp"] = lp_20ips_2016["Time stamp"].str.replace(r'[^0-9.: ]', '', regex=True).str.strip()
    lp_20ips_2016["Time stamp"] = pd.to_datetime(lp_20ips_2016.loc[:, "Time stamp"], format='%d.%m.%Y %H:%M:%S')

    lp_30ips_2017["Time stamp"] = lp_30ips_2017["Time stamp"].str.replace(r'[^0-9.: ]', '', regex=True).str.strip()
    lp_30ips_2017["Time stamp"] = pd.to_datetime(lp_30ips_2017.loc[:, "Time stamp"], format='%d.%m.%Y %H:%M:%S')

    train_2016_full = lp_20ips_2016.iloc[:, :-7]
    train_2017_full = lp_30ips_2017.iloc[:, :-8]
    test_2016_full = pd.concat([lp_20ips_2016["Time stamp"], lp_20ips_2016.iloc[:, -7:] ], axis=1)
    test_2017_full = pd.concat([lp_30ips_2017["Time stamp"], lp_30ips_2017.iloc[:, -8:] ], axis=1)

    train_2016_training_data = train_2016_full[train_2016_full["Time stamp"].dt.month <= 8]
    train_2016_val_data = train_2016_full[train_2016_full["Time stamp"].dt.month > 8]

    test_2016_training_data = test_2016_full[test_2016_full["Time stamp"].dt.month <= 8]
    test_2016_test_data = test_2016_full[test_2016_full["Time stamp"].dt.month > 8]

    train_2017_training_data = train_2017_full[train_2017_full["Time stamp"].dt.month <= 8]
    train_2017_val_data = train_2017_full[train_2017_full["Time stamp"].dt.month > 8]

    test_2017_training_data = test_2017_full[test_2017_full["Time stamp"].dt.month <= 8]
    test_2017_test_data = test_2017_full[test_2017_full["Time stamp"].dt.month > 8]

    train_2016_training_data.to_csv("tune/2016_train.csv", index=False)
    train_2016_val_data.to_csv("tune/2016_val.csv", index=False)
    test_2016_training_data.to_csv("test/2016_train.csv", index=False)
    test_2016_test_data.to_csv("test/2016_test.csv", index=False)

    train_2017_training_data.to_csv("tune/2017_train.csv", index=False)
    train_2017_val_data.to_csv("tune/2017_val.csv", index=False)
    test_2017_training_data.to_csv("test/2017_train.csv", index=False)
    test_2017_test_data.to_csv("test/2017_test.csv", index=False)
train_test_split()

In [2]:
prediction_length = 48
long_runtime = (3*60-5)*60 # in seconds
dev_runtime = 5*60 # in seconds

train_1 = pd.read_csv("tune/2016_train.csv")
train_2 = pd.read_csv("tune/2017_train.csv")
val_1 = pd.read_csv("tune/2016_val.csv")
val_2 = pd.read_csv("tune/2017_val.csv")

def prepare_AG_data(dfs: list[pd.DataFrame]):
    # Prepare Data
    melted_dfs = []
    for df in dfs:
        df = deepcopy(df)#[["Time stamp", "LG 1"]]
        df.loc[:, ["Time stamp"]] = df["Time stamp"].str.replace(r'[^0-9.: ]', '', regex=True).str.strip()
        df.loc[:, ["Time stamp"]] = pd.to_datetime(df["Time stamp"])
        # TODO Normalize Data else MASE means nothing
        # Concat Columns & Assign Ids
        df = df.melt(id_vars=['Time stamp'], var_name='item_id', value_name='target')
        df = df.dropna()
        df.interpolate(method='linear', inplace=True)
        melted_dfs.append(df)
    df = pd.concat(melted_dfs)
    df = TimeSeriesDataFrame.from_data_frame(
        df,
        id_column="item_id",
        timestamp_column="Time stamp"
    )
    return df

train = prepare_AG_data([train_1, train_2])
val = prepare_AG_data([val_1, val_2])
train

C:\Users\Jimmy\AppData\Local\Temp\ipykernel_39816\4077685799.py:21: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method='linear', inplace=True)
C:\Users\Jimmy\AppData\Local\Temp\ipykernel_39816\4077685799.py:21: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method='linear', inplace=True)
C:\Users\Jimmy\AppData\Local\Temp\ipykernel_39816\4077685799.py:21: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method='linear', inplace=True)
C:\Users\Jimmy\AppData\Local\Temp\ipykernel_39816\4077685799.py:21: FutureWarning: DataFrame.interpolate with object dtype is deprecated and w

target
item_id timestamp                  
LG 1    2016-01-01 00:15:00    39.0
        2016-01-01 00:30:00    41.0
        2016-01-01 00:45:00    39.0
        2016-01-01 01:00:00    40.0
        2016-01-01 01:15:00    39.0
...                             ...
LG 22   2017-08-31 22:45:00     1.5
        2017-08-31 23:00:00     1.6
        2017-08-31 23:15:00     1.4
        2017-08-31 23:30:00     1.0
        2017-08-31 23:45:00     0.7

[817706 rows x 1 columns]

In [3]:
# Custom Metric 
from sklearn.metrics import mean_squared_error 
from autogluon.timeseries.metrics import TimeSeriesScorer
class eRMSE(TimeSeriesScorer):

    equivalent_tabular_regression_metric = "root_mean_squared_error"

    def compute_metric(
        self, data_future: TimeSeriesDataFrame, predictions: TimeSeriesDataFrame, target: str = "target", **kwargs
    ) -> float:
        y_true, y_pred = self._get_point_forecast_score_inputs(data_future, predictions, target=target)
        # We always want to return a val (to do so not 85% of max but 95th percentile)
        ext_thr = np.percentile(y_true.dropna(), 95)
        mask = y_true > ext_thr
        val1 = np.sqrt(self._safemean((y_true[mask] - y_pred[mask]) ** 2))
        val2 = np.sqrt(self._safemean((y_true - y_pred) ** 2))
        return val1+val2

In [ ]:
# Model Exploration Exploration
# TODO Custom Config that compares all Models from Model Zoo
predictor = TimeSeriesPredictor(prediction_length=prediction_length, eval_metric=eRMSE, freq="15min")
predictor = predictor.fit(
    train,
    hyperparameters={
        "Chronos": {
            "model_path": "tiny",
            "batch_size": 64,
            "device": "cpu",
            "optimization_strategy": "openvino",
        },
        "NaiveModel": {},
        "SeasonalNaiveModel": {},
        "AverageModel": {},
        "SeasonalAverageModel": {},
        "ZeroModel": {},
        "ETSModel": {},
        #"AutoARIMAModel": {"n_jobs": -1},
        "AutoETSModel": {},
        "AutoCESModel": {},
        "ThetaModel": {},
        "ADIDAModel": {},
        #"CrostonClassicModel": {},
        #"CrostonOptimizedModel": {},
        "CrostonSBAModel": {},
        "IMAPAModel": {},
        "NPTSModel": {},
        "DeepARModel": {},
        "DLinearModel": {},
        "PatchTSTModel": {},
        "SimpleFeedForwardModel": {},
        "TemporalFusionTransformerModel": {},
        "TiDEModel": {},
        "WaveNetModel": {},
        "DirectTabularModel": {},
        "RecursiveTabularModel": {},
    },
    #verbosity=0,
    presets="best_quality",
    time_limit=long_runtime,
)
# TODO Investigate Leaderboard
print(predictor.evaluate(val))
predictor.leaderboard()

Beginning AutoGluon training... Time limit = 10500s
AutoGluon will save models to 'AutogluonModels\ag-20240912_093203'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          12
GPU Count:          0
Memory Avail:       4.67 GB / 15.34 GB (30.4%)
Disk Space Avail:   86.57 GB / 474.72 GB (18.2%)
Setting presets to: best_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': eRMSE,
 'freq': '15min',
 'hyperparameters': {'ADIDAModel': {},
                     'AutoCESModel': {},
                     'AutoETSModel': {},
                     'AverageModel': {},
                     'Chronos': {'batch_size': 64,
                                 'device': 'cpu',
                                 'model_path': 'tiny',
                                 'optimization_strategy': 'openvino'},
                     'CrostonSBAMode

In [ ]:
# TODO Tune best found Config